In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os 
import ast
import json
import time
from requests.exceptions import ConnectionError

https://stackoverflow.com/questions/62335537/scraping-text-from-kickstarter-projects-return-nothing

In [2]:
base = "/Users/ivankoh/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df_1 = df.drop_duplicates(subset=['id'])
df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df))
print(len(df_1))

392244
203510


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_41841/4110031066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])


In [30]:
def get_slug(url):
    return re.search('/projects/(.*)\?', url).group(1)

# get the slug from the urls. the slug is the input to the GraphQL API call 
slugs = list(map(get_slug, df_1['main_url'].tolist()))
# while iterating through all projects, add the story and risk to these lists. At the end len of both list 
# should be the same as the len of the dataframe. if all good then add as columns to the dataframe df. 
creator_list = []

In [ ]:
# instantiate a new Session 
s = requests.Session()

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

# PARAMS
counter = 0
# pause for 30 seconds every 5 iterations
for i in range(3000, 23511):
    slug = slugs[i]
    if counter == 5:
        print("TAKING A NAP ZZZ...")
        time.sleep(30)
        counter = 0
    counter += 1
    print(f"--------{slug}------")
    # POST request to GraphQL API situated in kickstarter domain 
    # response from POST request should contain the story and risks
    connect_try = 0
    not_found = True
    while connect_try < 30 and not_found:       
        try:
            r = s.post("https://www.kickstarter.com/graph",
                headers = {
                  'content-type': 'application/json',
                  'x-csrf-token': 'bL5nMRm5s147N8RxvNcth-E8NAOt3SprGYtvvKsVLnuZ3XwQ71cy1zrDfjIAGTCHV0obMRNqRMbBJT7nF2ZSAA',
                  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
                  'cookie': '_ksr_session=eus10UNK7iB%2FwIMXafXg5yJdYvRD7rHlR%2Ba1KEbUnB5EKTgQxNKEIqrp%2BIyjRC%2FNXw76u4Eki4AsiBom5M1LKNJKWCD65CBzeegWrk2%2FRV9Sht6dlOKplCATNdrmEIIa0Bt%2FKKGnT2m9ID1nTCduj73%2BdeJK663MNcbYWXBHkFyZJgUKh4JcCx3hL0UF2quVeBvfV45M0CNrT1j4DchvTlFBn%2BpE37ZaGldNz7BF0RAcCjIZrt%2FzNkxugCaFfWEPjxh9amOCvytIcdjxOaE68AAmHuU%3D--RPNmoWC%2FGnWTmsfw--9466UG%2Bqs2V%2F8B8zFtXUig%3D%3D'
                },
                json = {
                    "variables":{
                        "slug": slug
                    },
                    "query": "query CreatorSection($slug: String!) {\n  me {\n    id\n    name\n  }\n  project(slug: $slug) {\n    id\n    verifiedIdentity\n    creator {\n      id\n      name\n      imageUrl(width: 100)\n      url\n      lastLogin\n      biography\n      isFacebookConnected\n      allowsFollows\n      backingsCount\n      location {\n        displayableName\n      }\n      launchedProjects {\n        totalCount\n      }\n      websites {\n        url\n        domain\n      }\n    }\n    collaborators {\n      edges {\n        node {\n          name\n          imageUrl(width: 200)\n          url\n        }\n        title\n      }\n    }\n  }\n}\n"
                })
            not_found = False
        except ConnectionError:
            print("Connection Error")
            time.sleep(5)
            connect_try += 1
    try:
        result = r.json()
        print("-----CREATOR-----")
        creator_list.append(result['data']['project'])
    except:
        print(r)
        print("TAKING A BIG NAP ZZZ...")
        time.sleep(60*30)
        #try one more time
        r = s.post("https://www.kickstarter.com/graph",
                headers = {
                  'content-type': 'application/json',
                  'x-csrf-token': 'bL5nMRm5s147N8RxvNcth-E8NAOt3SprGYtvvKsVLnuZ3XwQ71cy1zrDfjIAGTCHV0obMRNqRMbBJT7nF2ZSAA',
                  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
                  'cookie': '_ksr_session=eus10UNK7iB%2FwIMXafXg5yJdYvRD7rHlR%2Ba1KEbUnB5EKTgQxNKEIqrp%2BIyjRC%2FNXw76u4Eki4AsiBom5M1LKNJKWCD65CBzeegWrk2%2FRV9Sht6dlOKplCATNdrmEIIa0Bt%2FKKGnT2m9ID1nTCduj73%2BdeJK663MNcbYWXBHkFyZJgUKh4JcCx3hL0UF2quVeBvfV45M0CNrT1j4DchvTlFBn%2BpE37ZaGldNz7BF0RAcCjIZrt%2FzNkxugCaFfWEPjxh9amOCvytIcdjxOaE68AAmHuU%3D--RPNmoWC%2FGnWTmsfw--9466UG%2Bqs2V%2F8B8zFtXUig%3D%3D'
                },
                json = {
                    "variables":{
                        "slug": slug
                    },
                    "query": "query CreatorSection($slug: String!) {\n  me {\n    id\n    name\n  }\n  project(slug: $slug) {\n    id\n    verifiedIdentity\n    creator {\n      id\n      name\n      imageUrl(width: 100)\n      url\n      lastLogin\n      biography\n      isFacebookConnected\n      allowsFollows\n      backingsCount\n      location {\n        displayableName\n      }\n      launchedProjects {\n        totalCount\n      }\n      websites {\n        url\n        domain\n      }\n    }\n    collaborators {\n      edges {\n        node {\n          name\n          imageUrl(width: 200)\n          url\n        }\n        title\n      }\n    }\n  }\n}\n"
                })
        try:
            result = r.json()
            print("-----CREATOR-----")
            creator_list.append(result['data']['project'])
        except:
            print("NAP DIDN't WORK", r)
            break

--------dewintermag/the-creation-of-a-digital-fashion-magazine------
-----CREATOR-----
--------daveholmes/kickstart-a-full-lengthbrazilian-influenced-album------
-----CREATOR-----
--------1525074813/patrick-hughes-second-cd-heirloom------
-----CREATOR-----
--------160246552/grand-h------
-----CREATOR-----
--------1864240616/the-power-of-protest------
-----CREATOR-----
TAKING A NAP ZZZ...
--------kindkoffee/kind-koffee------
-----CREATOR-----
--------612842963/the-true-survivors-of-suicide------
-----CREATOR-----
--------chayopdx/chayo-mexican-food-cart------
-----CREATOR-----
--------honeybeewaffleco/honey-bee-waffle-company------
-----CREATOR-----
--------littlelambmobileshop/little-lamb------
-----CREATOR-----
TAKING A NAP ZZZ...
--------brucieatx/brucie------
-----CREATOR-----
--------victorypencil/hidn-the-premium-leather-belt-with-a-secret-compartment------
-----CREATOR-----
--------covertproductsgroup/bat-tri-tool-multi-tool-coin-left-behind-by-aliens------
-----CREATOR-----
----

# Saving Data

## Latest Iteration Ran Till: 2000, Local copy latest: 1000 to 1999


In [29]:
#SAVE LIST IN LOCAL FOR USE LATER. NAME FILE AS {from}_{to]_{story/risk}.json
creator_file_name = "/Users/ivankoh/Downloads/bt4222 data/1000_1999_creator" #CHANGE THIS
creator_file_name += ".json"

with open(creator_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(creator_list, f, indent=2) 
    
# with open(story_file_name, 'r') as f:
#     story_subset = json.load(f)

In [27]:
print("creator:", len(creator_list)) 

creator: 1000


In [28]:
print("LATEST CALL DF ROW", "\n")
print(df_1.iloc[1000+len(creator_list)-1]['creator'])

print("LATEST CREATOR IN LIST", "\n")
print(creator_list[-1])


LATEST CALL DF ROW 

{"id":1224839424,"name":"Anders Nelson","slug":"tekione","is_registered":null,"is_email_verified":null,"chosen_currency":null,"is_superbacker":null,"avatar":{"thumb":"https://ksr-ugc.imgix.net/assets/025/499/255/9e8bb2c2b349a09a6773758ecd39c842_original.jpeg?ixlib=rb-4.0.2&w=40&h=40&fit=crop&v=1560628428&auto=format&frame=1&q=92&s=6b9a9b4bf109286c5b9b9b3536119837","small":"https://ksr-ugc.imgix.net/assets/025/499/255/9e8bb2c2b349a09a6773758ecd39c842_original.jpeg?ixlib=rb-4.0.2&w=80&h=80&fit=crop&v=1560628428&auto=format&frame=1&q=92&s=42c2bf2936a08e330eb58bf8eb68537f","medium":"https://ksr-ugc.imgix.net/assets/025/499/255/9e8bb2c2b349a09a6773758ecd39c842_original.jpeg?ixlib=rb-4.0.2&w=160&h=160&fit=crop&v=1560628428&auto=format&frame=1&q=92&s=edcb2a3ae3c1ff193abab2c0c74a993d"},"urls":{"web":{"user":"https://www.kickstarter.com/profile/tekione"},"api":{"user":"https://api.kickstarter.com/v1/users/1224839424?signature=1654834371.a9ceb3d9fdb59357ac04ad0478c020220e16a

In [165]:
s = requests.Session()

In [58]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "variables": {
      "slug": "thehermetichermit/the-transient-mind-tarot"
    },
    "query": "query CreatorSection($slug: String!) {\n  me {\n    id\n    name\n  }\n  project(slug: $slug) {\n    id\n    verifiedIdentity\n    creator {\n      id\n      name\n      imageUrl(width: 100)\n      url\n      lastLogin\n      biography\n      isFacebookConnected\n      allowsFollows\n      backingsCount\n      location {\n        displayableName\n      }\n      launchedProjects {\n        totalCount\n      }\n      websites {\n        url\n        domain\n      }\n    }\n    collaborators {\n      edges {\n        node {\n          name\n          imageUrl(width: 200)\n          url\n        }\n        title\n      }\n    }\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'fvjxrhwSYir6BG4U761oTRMXCYu4Ql88J4gTJaaXswCLm-qP6vzjo_vw1FdTY3VNpWEmuQb1MZH_JkJ-GuTPew',
  'user-agent': 'Mozilla/5.0 (Linux; U; Android 2.3.4; en-us; Kindle Fire Build/GINGERBREAD) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1',
  'cookie': '_ksr_session=XeMoOdlfAuvBg3vZkCPtR2KPkjnWtoE%2FGrwY68dNgE13OYvjkIxO0Eiaf4Oay2xzILiIHuLsmpCAH%2B9aBGT01%2B7gNDH6exw0dtPhWF5xVSYtZnJawqJNc8IUuF52GC%2FS7QHp1woosgy6PABnOt0X0ITzAvUEg8wwMmv5e1IBYNCmj8ZiCiUwlU2h08X3WX0lQSBSeatHjTEHV%2FgAjhqitXq1xvrNIY7MvvonoTMQat4mSbJKrOzGi74PUMwlhlgHz1oDCWy%2BqAiY%2BIQp9ft4c%2BK2ta8%3D--Rds4wbR2FagzmGoy--ZV4JbIYN8J%2FdS8uWWa5EQw%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json())
a = response.json()

200
[{'data': {'me': None, 'project': {'id': 'UHJvamVjdC03NTY5NDA0MDI=', 'verifiedIdentity': 'Alexandra Eastman', 'creator': {'id': 'VXNlci0yMDE4MTg3NDcy', 'name': 'Alex E.', 'imageUrl': 'https://ksr-ugc.imgix.net/assets/036/415/186/53fbf1ce977b0312aead1aee65648ff2_original.jpg?ixlib=rb-4.0.2&w=160&h=160&fit=crop&v=1662144462&auto=format&frame=1&q=92&s=ec5c2c4451e85b255cfc03bca9f88b37', 'url': 'https://www.kickstarter.com/profile/thehermetichermit', 'lastLogin': 1664301753, 'biography': 'American living on Canadian soil. \r\nTarot reader for almost 15 years, graphic designer, lover of life.', 'isFacebookConnected': False, 'allowsFollows': True, 'backingsCount': 0, 'location': {'displayableName': 'Bucharest, Romania'}, 'launchedProjects': {'totalCount': 2}, 'websites': []}, 'collaborators': {'edges': []}}}}]


In [65]:
print(a[0]['data']['project'])

{'id': 'UHJvamVjdC03NTY5NDA0MDI=', 'verifiedIdentity': 'Alexandra Eastman', 'creator': {'id': 'VXNlci0yMDE4MTg3NDcy', 'name': 'Alex E.', 'imageUrl': 'https://ksr-ugc.imgix.net/assets/036/415/186/53fbf1ce977b0312aead1aee65648ff2_original.jpg?ixlib=rb-4.0.2&w=160&h=160&fit=crop&v=1662144462&auto=format&frame=1&q=92&s=ec5c2c4451e85b255cfc03bca9f88b37', 'url': 'https://www.kickstarter.com/profile/thehermetichermit', 'lastLogin': 1664301753, 'biography': 'American living on Canadian soil. \r\nTarot reader for almost 15 years, graphic designer, lover of life.', 'isFacebookConnected': False, 'allowsFollows': True, 'backingsCount': 0, 'location': {'displayableName': 'Bucharest, Romania'}, 'launchedProjects': {'totalCount': 2}, 'websites': []}, 'collaborators': {'edges': []}}
